In [ ]:
# Installing Packages
!pip install selenium
!apt-get update
!apt install chromium-chromedriver
!pip install html5lib requests
!pip install pypixelmatch Pillow numpy
!pip install torch torchvision transformers pillow
!pip install opencv-python pytesseract nltk
!pip install google-cloud-vision openai requests pillow
!pip install Pillow pixelmatch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 476.0/476.0 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.7 MB/s eta 0:00:00
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [976 kB]
Ign:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Hit:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:11 https://p

In [ ]:
!pip install requests


In [ ]:
from google.cloud import vision
import openai
import requests
from PIL import Image
import io
import base64
from google.cloud import vision
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Part
from google.colab import drive
import shutil

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Global variables
OUTPUT_FOLDER = ""

# HTML template
html_template = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>{title}</title>
    <style>
        {generated_code}
    </style>
</head>
<body>
    {generated_body}
</body>
</html>
"""

In [ ]:
import shutil

source_path = '/content/drive/MyDrive/Dissertation Aditya Parmar : testing of Large Lang. Model/dissertation-19276544-536138fbae5d.json'
destination_path = '/content/dissertation-19276544-536138fbae5d.json'

# Copying the file /content/drive/MyDrive/Dissertation Aditya Parmar : testing of Large Lang. Model/dissertation-19276544-536138fbae5d.json
shutil.copy(source_path, destination_path)

'/content/dissertation-19276544-536138fbae5d.json'

In [ ]:
# setting Google crednetials up for Using Vision API.
!export GOOGLE_APPLICATION_CREDENTIALS="//content/dissertation-19276544-536138fbae5d.json"

import os
from google.cloud import vision


In [ ]:
# Setting up Google Cloud credentials
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/content/dissertation-19276544-536138fbae5d.json"

# Initialising Vertex AI
vertexai.init(project="dissertation-19276544", location="us-central1")

# Initialising the Vision API client
vision_client = vision.ImageAnnotatorClient()

In [ ]:
!gcloud auth login
!gcloud projects get-iam-policy dissertation-19276544

Go to the following link in your browser, and complete the sign-in prompts:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=32555940559.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fsdk.cloud.google.com%2Fauthcode.html&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&state=CSTvPx0eFYBZLFPzvqCX2iUJu0L51l&prompt=consent&token_usage=remote&access_type=offline&code_challenge=acjOanfGIRGvcOvFDhmk8O1OS5R_280J9IJn30y20Ag&code_challenge_method=S256

Once finished, enter the verification code provided in your browser: 4/0AQlEd8zTTicdydB90pjWQSqATl8gW1W-wH7CdXXs3RmwyLVatcbKBwgWGh5CjReNr1OcaQ

You are now logged in as [19276544@brookes.ac.uk].
Your current project

In [ ]:
!gcloud config set project dissertation-19276544

Updated property [core/project].


In [ ]:
def create_directory(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)
        print(f"Created directory: {directory}")
    else:
        print(f"Directory already exists: {directory}")

def log_memory_usage(batch_number, image_number):
    process = psutil.Process(os.getpid())
    memory_info = process.memory_info()
    with open(os.path.join(OUTPUT_FOLDER, 'memory_log.txt'), 'a') as f:
        f.write(f"Batch {batch_number}, Image {image_number}: {memory_info.rss / 1024 / 1024:.2f} MB\n")


In [ ]:
# this step detects the text in Images
def analyze_image(image_path):
    with open(image_path, 'rb') as image_file:
        content = image_file.read()

    image = vision.Image(content=content)
    response = vision_client.document_text_detection(image=image)

    return response

In [ ]:
# here HTML/CSS code is generated from Image analysis
def generate_html_css(image_analysis):
    api_key = ''  # Replace with your actual Gemini API key

    # Defining the endpoint URL for the Gemini FlashAPI
    url = 'https://api.gemini.com/v1/flashapi'  # Replace with the actual Gemini API endpoint

    # Defining the headers including your API key
    headers = {
        'Content-Type': 'application/json',
        'Authorization': f'Bearer {api_key}'
    }

    # Defining the payload
    payload = {
        'prompt': f"""
        Based on the following image analysis, generate HTML and CSS code:
        {image_analysis}

        Provide the HTML structure and CSS styles to recreate this layout.
        """,
        'max_tokens': 1000
    }

    # Making the API request
    response = requests.post(url, headers=headers, json=payload)

    # Checking the response
    if response.status_code == 200:
        return response.json()['choices'][0]['text'].strip()
    else:
        print(f'Failed with status code {response.status_code}')
        print(response.text)
        return None


In [ ]:
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        file_content = image_file.read()
        mime_type = "image/png" if image_path.lower().endswith(".png") else "image/jpeg"
        return mime_type, base64.b64encode(file_content).decode('utf-8')

In [ ]:
!pip install pixelmatch

In [ ]:


import os
import base64
import glob
import json
import time
import gc
import psutil
import tensorflow as tf
from tqdm import tqdm
from PIL import Image, ImageChops
import numpy as np
import matplotlib.pyplot as plt
import zipfile
import shutil
from google.cloud import vision
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Part
from google.colab import drive
import requests
from bs4 import BeautifulSoup
from skimage.metrics import structural_similarity as ssim
import pixelmatch.contrib.PIL as pixelmatch
import torch
from transformers import CLIPProcessor, CLIPModel
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

# Setting up Google Cloud credentials
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/content/dissertation-19276544-536138fbae5d.json"

# Initialising Vertex AI
vertexai.init(project="dissertation-19276544", location="us-central1")



def analyze_and_generate_code(image_path):
    mime_type, base64_image = encode_image(image_path)

    prompt = """
    Analyze this image of a web page and provide complete, functional HTML code to recreate it. Focus on:


        You are an expert web developer tasked with recreating a web page based on a single image. Your goal is to generate a pixel-perfect, fully functional HTML file with internal CSS that meticulously replicates the layout, styling, and functionality of the web page shown in the image. Follow these detailed instructions:

        ### 1. IMAGE ANALYSIS:
        - First, carefully analyze the provided image to determine its type:
            - **Wireframe sketch**: A low-fidelity design document outlining the basic structure of the application.
            - **Long screenshot of a website**: A full vertical capture of a web page, showing multiple sections.
            - **Mock-up or standard website screenshot**: A high-fidelity design or an actual web page screenshot.
        - Based on the image type, adjust your approach accordingly.

        ### 2. WIREFRAME HANDLING:
        - **Identify common wireframe elements**:
            - **Title Elements**: Rectangles with text labels. Convert these to appropriate <h1>, <h2>, etc. tags based on the visual hierarchy.
            - **Image Placeholders**: Squares or rectangles with an "X" inside. Convert these to <img> tags with placeholder images or <div> elements styled to represent image containers.
            - **Button Elements**: Small rectangles with text labels. Convert these to <button> or <a> tags.
            - **Input Fields**: Rectangles representing text input fields. Convert these to <input> or <textarea> tags.
            - **Paragraph Elements**: Rectangles with multiple horizontal lines. Convert these to <p> tags.
        - **No Content Hallucination**: Do not generate additional content beyond what is explicitly represented in the wireframe.

        ### 3. LONG SCREENSHOT HANDLING:
        - **Sectional Analysis**:
            - **Header Section**: Look for navigation bar, logo, and possibly a CTA button.
            - **Hero Section**: Often the first prominent section with large text, CTA, and background image/color.
            - **Feature or Service Sections**: Includes text, images, and CTAs describing offerings.
            - **Testimonials or Case Studies**: Sections with quotes, client logos, or case summaries.
            - **Footer Section**: Links, contact information, and legal disclaimers.
        - **Placeholder Images**: Recognize large black plus signs (+) in white boxes as image placeholders, and generate HTML structure accordingly.

        ### 4. MOCK-UP OR STANDARD WEBSITE SCREENSHOT HANDLING:
        - **Layout & Styling**:
            - Implement a clean and simple layout that follows the structure depicted in the image.
            - Use appropriate semantic HTML5 tags (e.g., <header>, <nav>, <main>, <footer>) to create a clear and accessible structure.
        - **Visual Styling**:
            - Extract and apply exact colors, fonts, and other styling details.
            - Match typography, including font families, sizes, weights, and line heights.

        ### 5. LAYOUT AND STRUCTURE:
        - **Responsive Layout**: Implement a responsive layout using CSS techniques (e.g., Flexbox, Grid) to ensure accurate positioning and spacing of all elements.

        ### 6. VISUAL STYLING:
        - **CSS Styling**:
            - Apply CSS that reflects the colors, typography, and visual effects from the image.
            - Replicate borders, rounded corners, box shadows, and hover states for interactive elements.

        ### 7. CONTENT AND TYPOGRAPHY:
        - **Text Reproduction**: Accurately reproduce all visible text content as it appears in the image.
        - **Text Alignment and Hierarchies**: Implement proper text alignment, spacing, and hierarchies (e.g., headings, paragraphs, lists).

        ### 8. INTERACTIVE ELEMENTS:
        - **Component Identification**: Identify and implement interactive components such as navigation menus, buttons, forms, and sliders.
        - **Hover and Active States**: Implement hover and active states to enhance usability.
        - **JavaScript**: Use minimal JavaScript if necessary for dropdowns, expandable sections, or other interactive elements.

        ### 9. IMAGES AND ICONS:
        - **Image Placement**: Use placeholder images with the same dimensions and add appropriate alt text.
        - **Icon Implementation**: Use icon fonts (e.g., Font Awesome) or SVGs to ensure scalability and performance.

        ### 10. RESPONSIVE DESIGN:
        - **Basic Responsiveness**: Implement basic responsive design principles to ensure the layout adapts to different screen sizes.

        ### 11. PERFORMANCE AND ACCESSIBILITY:
            - **CSS Optimization**: Group similar styles and use shorthand properties where appropriate.
            - **Accessibility**: Include ARIA attributes for improved accessibility where applicable.

        ### 12. CODE STRUCTURE AND COMMENTS:
            - **HTML Structure**: Provide a complete HTML5 structure with <!DOCTYPE html>, <html>, <head>, and <body> tags.
            - **Internal CSS**: Include all CSS within a <style> tag in the <head> section.
            - **Semantic Class Names**: Use clear, semantic class names that reflect the purpose or content of elements.
            - **Detailed Comments**: Add comments in both HTML and CSS to explain your implementation choices and any assumptions made.

        ### 13. OUTPUT:
        - **Final HTML File**: Output a single, complete HTML file that includes all necessary HTML, CSS, and minimal JavaScript (if required for functionality). Ensure that the code is clean, well-formatted, and adheres to modern web development best practices. The final result should, when opened in a web browser, look as close to the original image as possible.



    """

    # Initialising the Gemini model
    model = GenerativeModel("gemini-1.5-flash")
    # use "gemini-1.5-pro" for Pro
    # use "gemini-1.5-flash" for Flash

    # Creating  the request and get the response
    response = model.generate_content(
        [
            Part.from_text(prompt),
            Part.from_data(data=base64_image, mime_type=mime_type)
        ],
        generation_config={
            "max_output_tokens": 2048,
            "temperature": 0.4,
            "top_p": 1,
            "top_k": 32
        }
    )

    # Extracting  the text from the response
    return response.text




In [ ]:
def validate_html(html_content):
    validator_url = "https://validator.w3.org/nu/?out=json"
    headers = {
        'Content-Type': 'text/html; charset=utf-8',
        'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2272.101 Safari/537.36'
    }
    response = requests.post(validator_url, headers=headers, data=html_content.encode('utf-8'))

    if response.status_code == 200:
        result = response.json()
        errors = [msg for msg in result['messages'] if msg['type'] == 'error']
        warnings = [msg for msg in result['messages'] if msg['type'] == 'info']
        return len(errors) == 0, len(errors), len(warnings)
    else:
        return False, -1, -1

In [ ]:
def analyze_html(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')
    nav_menu = soup.find('nav')
    nav_location = "Not found" if not nav_menu else nav_menu.parent.name
    sections = soup.find_all('section')
    section_count = len(sections)
    main_headline = soup.find('h1')

    headline_text = main_headline.text.strip() if main_headline else "No main headline found"
    footer = soup.find('footer')
    footer_content = footer.text.strip() if footer else "No footer found"
    images = soup.find_all('img')
    image_count = len(images)
    image_captions = [img.get('alt', 'No caption') for img in images]
    links = soup.find_all('a')
    hyperlinks = [{"text": link.text.strip(), "url": link.get('href', 'No URL')} for link in links]

    return {
        "nav_menu_location": nav_location,
        "section_count": section_count,
        "main_headline": headline_text,
        "footer_content": footer_content,
        "image_count": image_count,
        "image_captions": image_captions,
        "hyperlinks": hyperlinks
    }

In [ ]:
# semantic evaluation of generated HTML Code
def evaluate_semantic_tags(html_content):
    expected_tags = ['header', 'nav', 'main', 'article', 'section', 'aside', 'footer']
    soup = BeautifulSoup(html_content, 'html.parser')
    used_tags = [tag.name for tag in soup.find_all()]
    correctly_used_tags = [tag for tag in used_tags if tag in expected_tags]
    precision = len(correctly_used_tags) / len(used_tags) if used_tags else 0
    recall = len(correctly_used_tags) / len(expected_tags)
    return precision, recall, correctly_used_tags, [tag for tag in used_tags if tag not in expected_tags]


In [ ]:
# Comparing input images with GUI generated images
def compare_images(image1_path, image2_path):
    image1 = Image.open(image1_path)
    image2 = Image.open(image2_path)
    if image1.size != image2.size:
        image2 = image2.resize(image1.size)
    diff_image = Image.new("RGBA", image1.size)
    mismatch = pixelmatch.pixelmatch(image1, image2, diff_image)
    total_pixels = image1.size[0] * image1.size[1]
    diff_percentage = (mismatch / total_pixels) * 100
    return mismatch, diff_percentage, diff_image

In [ ]:
# Function for calculating SSIM score for Evalution
def calculate_ssim(image1_path, image2_path):
    img1 = Image.open(image1_path).convert('L')
    img2 = Image.open(image2_path).convert('L')
    img1 = np.array(img1)
    img2 = np.array(img2)
    if img1.shape != img2.shape:
        img2 = np.array(Image.fromarray(img2).resize(img1.shape[::-1]))
    return ssim(img1, img2)

# Function for calculating CLIP score for evaluation
def calculate_clip_score(image1_path, image2_path):
    model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
    processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
    image1 = Image.open(image1_path).convert('RGB')
    image2 = Image.open(image2_path).convert('RGB')
    inputs = processor(images=[image1, image2], return_tensors="pt", padding=True)
    with torch.no_grad():
        image_features = model.get_image_features(**inputs)
    similarity = torch.nn.functional.cosine_similarity(image_features[0], image_features[1], dim=0)
    return similarity.item()

In [ ]:
# Function for capturing a screenshot
def capture_screenshot(html_path, output_path):
    options = Options()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome(options=options)
    driver.get(f'file://{html_path}')
    driver.save_screenshot(output_path)
    driver.quit()

In [ ]:
# Processing single image
def process_single_image(image_path, output_folder):
    image_name = os.path.splitext(os.path.basename(image_path))[0]
    print(f"Processing {image_name}")

    try:
        # Checking image dimensions so that exceeding dimension are skipped
        with Image.open(image_path) as img:
            width, height = img.size
            aspect_ratio = height / width
            if width > 3000 or height > 40000 or aspect_ratio > 20:
                print(f"Skipping {image_name} due to excessive width, height, or aspect ratio")
                return None

        # Generating the code
        try:
            generated_code = analyze_and_generate_code(image_path)
        except Exception as e:
            print(f"Error generating code for {image_name}: {str(e)}")
            return None

        # Extracting the CSS and body from generated code
        css_start = generated_code.find('<style>')
        css_end = generated_code.find('</style>')
        body_start = generated_code.find('<body>')
        body_end = generated_code.find('</body>')

        css = generated_code[css_start+7:css_end] if css_start != -1 and css_end != -1 else ''
        body = generated_code[body_start+6:body_end] if body_start != -1 and body_end != -1 else generated_code

        # Creating HTML content
        html_content = html_template.format(
            title=image_name,
            generated_code=css,
            generated_body=body
        )

        # Saving individual HTML file
        output_file_path = os.path.join(output_folder, f"{image_name}.html")
        with open(output_file_path, 'w', encoding='utf-8') as f:
            f.write(html_content)

        # Capturing screenshot
        screenshot_path = os.path.join(output_folder, f"{image_name}_screenshot.png")
        try:
            capture_screenshot(output_file_path, screenshot_path)
        except Exception as e:
            print(f"Error capturing screenshot for {image_name}: {str(e)}")
            screenshot_path = image_path  # Use original image if screenshot fails

        # Calculating metrics
        try:
            ssim_score = calculate_ssim(image_path, screenshot_path)
            _, diff_percentage, diff_image = compare_images(image_path, screenshot_path)
            clip_score = calculate_clip_score(image_path, screenshot_path)
            is_valid, _, _ = validate_html(html_content)
            precision, recall, _, _ = evaluate_semantic_tags(html_content)
        except Exception as e:
            print(f"Error calculating metrics for {image_name}: {str(e)}")
            ssim_score = diff_percentage = clip_score = precision = recall = 0
            is_valid = False
            diff_image = Image.new('RGB', (100, 100))  # Create a blank image if comparison fails

        # Saving difference image
        diff_image_path = os.path.join(output_folder, f"{image_name}_diff.png")
        diff_image.save(diff_image_path)

        # Collecting results
        image_results = {
            'Image': image_name,
            'SSIM': ssim_score,
            'PixelMatch_DiffPercentage': diff_percentage,
            'CLIP': clip_score,
            'HTML_Valid': is_valid,
            'Semantic_Precision': precision,
            'Semantic_Recall': recall,
        }

        return image_results

    except Exception as e:
        print(f"Unexpected error processing {image_name}: {str(e)}")
        return None


In [ ]:

def process_batch(image_paths, output_folder, batch_number):
    results = []
    skipped_images = []

    for i, image_path in enumerate(tqdm(image_paths, desc=f"Processing Batch {batch_number}")):
        result = process_single_image(image_path, output_folder)
        if result:
            results.append(result)
        else:
            skipped_images.append(os.path.basename(image_path))

        time.sleep(60)

        # Logging memory usage
        log_memory_usage(batch_number, i+1)

    # Displaying summary after processing the entire batch
    print(f"\nBatch {batch_number} Processing Summary:")
    print(f"Total images processed: {len(results)}")
    print(f"Skipped images: {len(skipped_images)}")
    if skipped_images:
        print("Skipped image names:")
        for img in skipped_images:
            print(f"- {img}")

    if results:
        print("Average Scores:")
        avg_scores = {k: sum(r[k] for r in results if k in r) / len(results) for k in results[0] if k != 'Image'}
        for key, value in avg_scores.items():
            print(f"Avg {key}: {value:.4f}")
    else:
        print("No images were successfully processed in this batch.")

    return results, skipped_images


def save_checkpoint(checkpoint_file, processed_items):
    with open(checkpoint_file, 'w') as f:
        json.dump({'processed_items': processed_items}, f)

def load_checkpoint(checkpoint_file):
    if os.path.exists(checkpoint_file):
        with open(checkpoint_file, 'r') as f:
            return json.load(f)['processed_items']
    return []

def enqueue_tasks(individual_folder, zip_folder, output_folder, checkpoint_file):
    all_images = glob.glob(os.path.join(individual_folder, "*.png"))
    all_zips = glob.glob(os.path.join(zip_folder, "*.zip"))
    processed_items = load_checkpoint(checkpoint_file)

    for image_path in all_images:
        if image_path not in processed_items:
            queue.enqueue(process_single_image, image_path, output_folder)

    for zip_path in all_zips:
        if zip_path not in processed_items:
            queue.enqueue(process_zip, zip_path, output_folder)

    return len(all_images) + len(all_zips) - len(processed_items)

### Input & Output

In [ ]:
def main_processing():
    images_folder = input("Enter the path to the folder with images: ").strip()
    output_folder = input("Enter the path to the output folder: ").strip()

    create_directory(output_folder)
    checkpoint_file = os.path.join(output_folder, "checkpoint.json")
    error_log_file = os.path.join(output_folder, "error_log.txt")

    try:
        processed_items = load_checkpoint(checkpoint_file)
        all_images = glob.glob(os.path.join(images_folder, "*.*"))
        all_images = [img for img in all_images if img.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp'))]

        print(f"Found {len(all_images)} images.")
        print(f"Already processed: {len(processed_items)} images.")

        # Process images one by one
        for i, image_path in enumerate(all_images):
            if image_path in processed_items:
                continue

            try:
                result = process_single_image(image_path, output_folder)

                if result:
                    # Save result for this image
                    with open(os.path.join(output_folder, f"image_{i+1}_result.json"), 'w') as f:
                        json.dump(result, f)

                    processed_items.append(image_path)
                    save_checkpoint(checkpoint_file, processed_items)
                else:
                    # Log skipped image
                    with open(error_log_file, 'a') as f:
                        f.write(f"Skipped image: {image_path}\n")

            except Exception as e:
                print(f"Error processing image {image_path}: {str(e)}")
                with open(error_log_file, 'a') as f:
                    f.write(f"Error processing image {image_path}: {str(e)}\n")

            # Timer of 60 seconds before processing the next image for maintaining the quota limits of the Gemini AI API
            if i < len(all_images) - 1:
                time.sleep(60)

        print("All images processed!")

        # Combining all results into a single JSONL file
        final_results_file = os.path.join(output_folder, 'all_results.jsonl')
        with open(final_results_file, 'w') as outfile:
            for i in range(1, len(all_images) + 1):
                result_file = os.path.join(output_folder, f"image_{i}_result.json")
                if os.path.exists(result_file):
                    with open(result_file, 'r') as infile:
                        outfile.write(infile.read() + '\n')

        print(f"All results combined in {final_results_file}")
        print(f"Error log saved to {error_log_file}")

    except Exception as e:
        print(f"An error occurred: {str(e)}")
        print("Please check your input and try again.")

# Printing the scores
def show_results(output_folder):
    results_file = os.path.join(output_folder, 'all_results.jsonl')
    if not os.path.exists(results_file):
        print(f"No results found in {output_folder}")
        return

    with open(results_file, 'r') as f:
        results = [json.loads(line) for line in f]

    print(f"Total images processed: {len(results)}")
    print("\nAverage Scores:")
    avg_scores = {k: sum(r[k] for r in results if k in r) / len(results) for k in results[0] if k != 'Image'}
    for key, value in avg_scores.items():
        print(f"Avg {key}: {value:.4f}")

    print("\nDetailed Results:")
    for result in results:
        print(f"\nImage: {result['Image']}")
        print(f"SSIM: {result['SSIM']:.4f}")
        print(f"PixelMatch Difference: {result['PixelMatch_DiffPercentage']:.2f}%")
        print(f"CLIP Score: {result['CLIP']:.4f}")
        print(f"HTML Valid: {'Yes' if result['HTML_Valid'] else 'No'}")
        print(f"Semantic Precision: {result['Semantic_Precision']:.4f}")
        print(f"Semantic Recall: {result['Semantic_Recall']:.4f}")
        print("-" * 50)

if __name__ == "__main__":
    main_processing()
    # show_results("/content/drive/MyDrive/Dissertation Aditya Parmar : testing of Large Lang. Model/The Output Folder/Gemini_Final1000")

Enter the path to the folder with images: /content/drive/MyDrive/Dissertation Aditya Parmar : testing of Large Lang. Model/Final Folder with all four datasets
Enter the path to the output folder: /content/drive/MyDrive/Dissertation Aditya Parmar : testing of Large Lang. Model/1000_zip_ten
Directory already exists: /content/drive/MyDrive/Dissertation Aditya Parmar : testing of Large Lang. Model/1000_zip_ten
Found 1024 images.
Already processed: 1015 images.
Processing forexdemo.info


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Processing examinerlive.co.uk
Skipping examinerlive.co.uk due to excessive width, height, or aspect ratio
Processing ekantipur.com
Error generating code for ekantipur.com: Cannot get the response text.
Cannot get the Candidate text.
Response candidate content has no parts (and thus no text). The candidate is likely blocked by the safety filters.
Content:
{}
Candidate:
{
  "finish_reason": "RECITATION",
  "safety_ratings": [
    {
      "category": "HARM_CATEGORY_HATE_SPEECH",
      "probability": "NEGLIGIBLE",
      "probability_score": 0.15039062,
      "severity": "HARM_SEVERITY_NEGLIGIBLE",
      "severity_score": 0.11279297
    },
    {
      "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
      "probability": "NEGLIGIBLE",
      "probability_score": 0.11425781,
      "severity": "HARM_SEVERITY_LOW",
      "severity_score": 0.20800781
    },
    {
      "category": "HARM_CATEGORY_HARASSMENT",
      "probability": "NEGLIGIBLE",
      "probability_score": 0.15917969,
      "severity":

/usr/local/lib/python3.10/dist-packages/PIL/Image.py:3167: DecompressionBombWarning: Image size (108872225 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


Streaming output truncated to the last 5000 lines.
CLIP Score: 0.5557
HTML Valid: Yes
Semantic Precision: 0.1724
Semantic Recall: 0.7143
--------------------------------------------------

Image: sketch_70
SSIM: 0.8460
PixelMatch Difference: 7.76%
CLIP Score: 0.6536
HTML Valid: Yes
Semantic Precision: 0.1613
Semantic Recall: 0.7143
--------------------------------------------------

Image: sketch_150
SSIM: 0.6669
PixelMatch Difference: 21.20%
CLIP Score: 0.8453
HTML Valid: Yes
Semantic Precision: 0.2143
Semantic Recall: 0.4286
--------------------------------------------------

Image: sketch_144
SSIM: 0.8444
PixelMatch Difference: 7.44%
CLIP Score: 0.7368
HTML Valid: Yes
Semantic Precision: 0.0000
Semantic Recall: 0.0000
--------------------------------------------------

Image: sketch_64
SSIM: 0.6910
PixelMatch Difference: 15.14%
CLIP Score: 0.6240
HTML Valid: Yes
Semantic Precision: 0.2000
Semantic Recall: 1.0000
--------------------------------------------------

Image: sketch_178
S

# THE END

In [ ]:
# Code for generating a CSV file that holds score of all files from their JSON files
import os
import json
import csv
import glob
from tqdm import tqdm

def jsonl_to_csv(input_folder, output_path):
    # Get all .jsonl files in the input folder
    jsonl_files = glob.glob(os.path.join(input_folder, '*.jsonl'))

    # If output_path is a directory, create a file name
    if os.path.isdir(output_path):
        output_csv = os.path.join(output_path, 'image_metrics_output.csv')
    else:
        output_csv = output_path

    # Ensure the directory exists
    os.makedirs(os.path.dirname(output_csv), exist_ok=True)

    # Prepare CSV writer
    with open(output_csv, 'w', newline='') as csvfile:
        fieldnames = ['Image Name']
        csv_writer = None  # We'll initialize this after we know all the fields

        # Process each .jsonl file
        for jsonl_file in tqdm(jsonl_files, desc="Processing JSONL files"):
            with open(jsonl_file, 'r') as file:
                for line in file:
                    data = json.loads(line)

                    # If this is the first row, set up the CSV writer
                    if csv_writer is None:
                        fieldnames.extend([key for key in data.keys() if key != 'Image'])
                        csv_writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
                        csv_writer.writeheader()

                    # Prepare the row data
                    row_data = {'Image Name': data.get('Image', 'Unknown')}
                    row_data.update({key: value for key, value in data.items() if key != 'Image'})

                    # Write the row
                    csv_writer.writerow(row_data)

    print(f"CSV file created: {output_csv}")

# Usage
input_folder = input("Enter the path to the folder containing .jsonl files: ").strip()
output_path = input("Enter the path for the output CSV file or directory: ").strip()

jsonl_to_csv(input_folder, output_path)

Enter the path to the folder containing .jsonl files: /content/drive/MyDrive/Dissertation Aditya Parmar : testing of Large Lang. Model/1000_zip_ten
Enter the path for the output CSV file or directory: /content/drive/MyDrive/Dissertation Aditya Parmar : testing of Large Lang. Model/1000_zip_ten


Processing JSONL files: 100%|██████████| 7/7 [00:01<00:00,  4.05it/s]

CSV file created: /content/drive/MyDrive/Dissertation Aditya Parmar : testing of Large Lang. Model/1000_zip_ten/image_metrics_output.csv


In [ ]:
# Convert JSONL to CSV
import json
import csv
import os
from google.colab import files

def read_jsonl(file_path):
    with open(file_path, 'r') as file:
        return [json.loads(line) for line in file]

def convert_jsonl_to_csv(jsonl_file, csv_file):
    data = read_jsonl(jsonl_file)

    if not data:
        print("No data found in the JSONL file.")
        return

    fieldnames = list(data[0].keys())

    with open(csv_file, 'w', newline='') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()

        for item in data:
            writer.writerow(item)

    print(f"\nAll data has been written to {csv_file}")

def download_file(file_path):
    files.download(file_path)

# Usage
jsonl_file = "/content/drive/MyDrive/Dissertation Aditya Parmar : testing of Large Lang. Model/1000_zip_ten/esi-group.com_diff.png"
csv_file = "/content/drive/MyDrive/Dissertation Aditya Parmar : testing of Large Lang. Model/The Output Folder/Gemini_Final1000/"

# Convert JSONL to CSV
convert_jsonl_to_csv(jsonl_file, csv_file)

# Download the CSV file
print("\nInitiating download of the CSV file...")
download_file(csv_file)

In [ ]:

import google.generativeai as genai

genai.configure(api_key='')

# List available models
for m in genai.list_models():
    if 'generateContent' in m.supported_generation_methods:
        print(m.name)

# Select a model
model = genai.GenerativeModel('gemini-pro')

# Generate content
response = model.generate_content("Hello, Gemini!")
print(response.text)

In [ ]:
def format_html(css_content):
    # Wrap the CSS content in a style tag
    formatted_result = f"""
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Styled Result</title>
    <style>
{css_content}
    </style>
</head>
<body>
    <div id="content">
        <!-- Your content will go here -->
    </div>
</body>
</html>
"""
    return formatted_result

formatted_result = format_html(result)

In [ ]:
def save_locally(content, filename):
    with open(filename, 'w', encoding='utf-8') as f:
        f.write(content)
    print(f"File saved locally as {filename}")

def save_to_drive(content, filename, drive_path):
    # Mount Google Drive
    drive.mount('/content/drive')

    # Construct the full path
    full_path = os.path.join('/content/drive/My Drive', drive_path, filename)

    # Ensure the directory exists
    os.makedirs(os.path.dirname(full_path), exist_ok=True)

    # Save the file
    with open(full_path, 'w', encoding='utf-8') as f:
        f.write(content)
    print(f"File saved to Google Drive as {full_path}")



In [ ]:
!pip install selenium
!apt-get update
!apt install chromium-chromedriver

In [ ]:
from openai import OpenAI
import base64
import os
from google.colab import drive, files
from IPython.display import HTML, display, IFrame
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import time

In [ ]:

def open_local_file_in_colab(filename):
    with open(filename, 'r', encoding='utf-8') as f:
        content = f.read()
    display(HTML(content))
    print(f"Local file {filename} opened in Colab output cell")

def open_drive_file_in_colab(drive_path, filename):
    full_path = os.path.join(drive_path, filename)
    with open(full_path, 'r', encoding='utf-8') as f:
        content = f.read()
    display(HTML(content))
    print(f"Drive file {full_path} opened in Colab output cell")

def open_in_new_tab(file_path):
    js = f'''
    var win = window.open('{file_path}', '_blank');
    win.focus();
    '''
    display(Javascript(js))
    print(f"File opened in a new tab: {file_path}")




In [ ]:
def save_to_drive_and_get_link(content, filename, drive_path):
    full_path = os.path.join('/content/drive/MyDrive/Dissertation Aditya Parmar : testing of Large Lang. Model', drive_path, filename)
    os.makedirs(os.path.dirname(full_path), exist_ok=True)
    with open(full_path, 'w', encoding='utf-8') as f:
        f.write(content)
    print(f"File saved to Google Drive as {full_path}")

    # Create a shareable link
    from google.colab import auth
    from googleapiclient.discovery import build
    auth.authenticate_user()
    drive_service = build('drive', 'v3')
    file_id = drive_service.files().list(q=f"name='{filename}' and '{drive_path}' in parents", fields="files(id)").execute().get('files', [])[0]['id']
    drive_service.permissions().create(fileId=file_id, body={"role": "reader", "type": "anyone"}).execute()
    link = f"https://drive.google.com/file/d/{file_id}/view?usp=sharing"
    print(f"Shareable link: {link}")
    return link

def take_screenshot(url, output_path):
    chrome_options = Options()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    chrome_options.add_argument('--window-size=1920,1080')

    driver = webdriver.Chrome(options=chrome_options)
    driver.get(url)

    # Wait for the page to load
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, "body")))

    # Scroll to capture full page
    total_height = driver.execute_script("return document.body.scrollHeight")
    driver.set_window_size(1920, total_height)
    time.sleep(2)  # Wait for any dynamic content to load

    driver.save_screenshot(output_path)
    driver.quit()
    print(f"Screenshot saved as {output_path}")


In [ ]:
# from google.colab import auth
# auth.authenticate_user()
!pip install PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive

# Authenticate and create the PyDrive client
gauth = GoogleAuth()
gauth.LocalWebserverAuth()  # Opens a web-based authentication dialog
drive = GoogleDrive(gauth)


In [ ]:
import os

# Usage
image_path = "/content/drive/MyDrive/Dissertation Aditya Parmar : testing of Large Lang. Model/dataset_images/pinellascounty.org.png"
image_name = os.path.splitext(os.path.basename(image_path))[0]
result = analyze_and_generate_code(image_path)
formatted_result = format_html(result)

In [ ]:
import base64
from IPython.display import display, HTML
from google.colab import files

In [ ]:
import urllib.parse

# Save locally
local_filename = f'result-{image_name}.html'
with open(local_filename, 'w', encoding='utf-8') as f:
    f.write(formatted_result)

print(f"File saved locally as: {local_filename}")

# Create a data URL
data_url = f"data:text/html,{urllib.parse.quote(formatted_result)}"

# Create a clickable link that opens in a new tab
display(HTML(f'<a href="{data_url}" target="_blank">Click here to open the styled HTML in a new tab</a>'))

# You can still provide a download option if needed
from google.colab import files
files.download(local_filename)


In [ ]:
import os
import subprocess

# Assuming you've already saved the file as before
local_filename = f'result-{image_name}.html'

# Get the full path of the file
file_path = os.path.abspath(local_filename)

# Determine the operating system
if os.name == 'nt':  # Windows
    # Command for Windows
    command = f'start chrome "{file_path}"'

    # Open Command Prompt and execute the command
    try:
        subprocess.run(command, shell=True, check=True)
    except subprocess.CalledProcessError as e:
        print(f"Error opening {file_path} in Google Chrome: {e}")

elif os.name == 'posix':  # macOS or Linux
    # Command for macOS (also works for most Linux distributions)
    command = f'google-chrome "{file_path}"'

    # For macOS, you might need to use the following instead:
    # command = f'open -a "Google Chrome" "{file_path}"'

    # Open Terminal and execute the command
    try:
        subprocess.run(command, shell=True, check=True)
    except subprocess.CalledProcessError as e:
        print(f"Error opening {file_path} in Google Chrome: {e}")

else:
    print("Unsupported operating system")

print(f"Attempting to open {file_path} in Google Chrome")


In [ ]:
import os
import subprocess
from IPython.display import HTML, display

# Assuming you've already saved the file as before
local_filename = f'result-{image_name}.html'

# Get the full path of the file
file_path = os.path.abspath(local_filename)

# Check if we're in a Jupyter environment
try:
    get_ipython()
    in_notebook = True
except NameError:
    in_notebook = False

if in_notebook:
    # If in a notebook (like Colab), display the HTML content
    with open(file_path, 'r') as file:
        html_content = file.read()
    display(HTML(html_content))
    print(f"Displaying HTML content from {file_path}")
else:
    # If on a local machine, try to open in browser
    if os.name == 'nt':  # Windows
        command = f'start chrome "{file_path}"'
    elif os.name == 'posix':  # macOS or Linux
        if os.uname().sysname == 'Darwin':  # macOS
            command = f'open -a "Google Chrome" "{file_path}"'
        else:  # Linux
            command = f'xdg-open "{file_path}"'
    else:
        print("Unsupported operating system")
        exit(1)

    try:
        subprocess.run(command, shell=True, check=True)
        print(f"Opened {file_path} in default browser")
    except subprocess.CalledProcessError as e:
        print(f"Error opening {file_path}: {e}")
        print("HTML content:")
        with open(file_path, 'r') as file:
            print(file.read())

In [ ]:
import os
from IPython.display import HTML, display
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from PIL import Image
import io
import base64

# Assuming you've already saved the HTML file
local_filename = f'result-{image_name}.html'
file_path = os.path.abspath(local_filename)

# 1. Display the HTML in the notebook
with open(file_path, 'r') as file:
    html_content = file.read()
display(HTML(html_content))

# 2. Capture a screenshot of the rendered HTML
# Set up Chrome options for headless browsing
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

# Create a new Chrome driver
driver = webdriver.Chrome(options=chrome_options)

# Navigate to the local HTML file
driver.get(f"file://{file_path}")

# Capture the full page screenshot
screenshot = driver.get_screenshot_as_png()

# Close the driver
driver.quit()

# Convert the screenshot to a PIL Image
screenshot_image = Image.open(io.BytesIO(screenshot))

# Save the screenshot
screenshot_path = "rendered_html_screenshot.png"
screenshot_image.save(screenshot_path)

print(f"Screenshot saved as {screenshot_path}")

# 3. If you need to interact with the rendered content using Selenium,
# you can do so here before taking the screenshot

# 4. Compare the screenshot with the original input image
# Assuming you have the original input image path
original_image_path = "/content/drive/MyDrive/Dissertation Aditya Parmar : testing of Large Lang. Model/dataset_images/00web.net.png"
original_image = Image.open(original_image_path)

# Resize images to the same dimensions for comparison
max_width = max(screenshot_image.width, original_image.width)
max_height = max(screenshot_image.height, original_image.height)

screenshot_image_resized = screenshot_image.resize((max_width, max_height))
original_image_resized = original_image.resize((max_width, max_height))

# Compare images pixel by pixel
diff = Image.new("RGB", (max_width, max_height))


for x in range(max_width):
    for y in range(max_height):
        r1, g1, b1 = screenshot_image_resized.getpixel((x, y))
        # Handle different image modes for original image
        pixel_values = original_image_resized.getpixel((x, y))
        if len(pixel_values) == 3:
            r2, g2, b2 = pixel_values
        elif len(pixel_values) == 4:  # Handle RGBA images
            r2, g2, b2, _ = pixel_values
        # Add handling for other modes if necessary
        diff.putpixel((x, y), (abs(r1-r2), abs(g1-g2), abs(b1-b2)))


# Save the difference image
diff_path = "difference_image.png"
diff.save(diff_path)

print(f"Difference image saved as {diff_path}")

# Display the screenshot and difference image in the notebook
from IPython.display import Image as IPImage

display(IPImage(filename=screenshot_path))
display(IPImage(filename=diff_path))

In [ ]:
import os
from IPython.display import HTML, display
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from PIL import Image
import io
import numpy as np
from skimage.metrics import structural_similarity as ssim
from skimage.transform import resize
import matplotlib.pyplot as plt

# Assuming you've already saved the HTML file
local_filename = f'result-{image_name}.html'
file_path = os.path.abspath(local_filename)

# 1. Display the HTML in the notebook
with open(file_path, 'r') as file:
    html_content = file.read()
display(HTML(html_content))

# 2. Capture a screenshot of the rendered HTML
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--start-maximized")  # Starts the browser maximized

driver = webdriver.Chrome(options=chrome_options)

try:
    # Navigating to the local HTML file
    driver.get(f"file://{file_path}")

    # Waiting for the page to load
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, "body")))

    # Getting the total height of the page
    total_height = driver.execute_script("return document.body.parentNode.scrollHeight")

    # Setting the viewport size to capture the full width and the calculated height
    driver.set_window_size(1920, total_height)

    # Capturing the full page screenshot
    screenshot = driver.get_screenshot_as_png()

except Exception as e:
    print(f"An error occurred: {e}")
    driver.quit()
    raise

finally:
    driver.quit()

# Converting the screenshot to a PIL Image
screenshot_image = Image.open(io.BytesIO(screenshot))

# Saving the screenshot
screenshot_path = "full_page_screenshot.png"
screenshot_image.save(screenshot_path)

print(f"Full page screenshot saved as {screenshot_path}")

def get_manual_scores():
    criteria = [
        "Content Accuracy",
        "Layout Structure",
        "Visual Elements",
        "Styling",
        "Functionality Indicators"
    ]
    scores = {}
    total_score = 0

    print("Please rate each criterion from 0-3:")
    for criterion in criteria:
        while True:
            try:
                score = int(input(f"{criterion} (0-3): "))
                if 0 <= score <= 3:
                    scores[criterion] = score
                    total_score += score
                    break
                else:
                    print("Please enter a score between 0 and 3.")
            except ValueError:
                print("Please enter a valid integer.")

    return scores, total_score

def get_rating(total_score):
    if 13 <= total_score <= 15:
        return "Excellent"
    elif 10 <= total_score <= 12:
        return "Good"
    elif 7 <= total_score <= 9:
        return "Fair"
    elif 4 <= total_score <= 6:
        return "Poor"
    else:
        return "Failing"


# Comparing the screenshot with the original input image
original_image_path = "/content/drive/MyDrive/Dissertation Aditya Parmar : testing of Large Lang. Model/dataset_images/pinellascounty.org.png"
original_image = Image.open(original_image_path)

# Converting images to grayscale numpy arrays
screenshot_gray = np.array(screenshot_image.convert('L'))
original_gray = np.array(original_image.convert('L'))

# Resizing images to the same dimensions for comparison
min_shape = tuple(min(s1, s2) for s1, s2 in zip(screenshot_gray.shape, original_gray.shape))
screenshot_resized = resize(screenshot_gray, min_shape, anti_aliasing=True)
original_resized = resize(original_gray, min_shape, anti_aliasing=True)

# Computing SSIM between the two images
ssim_score, diff = ssim(screenshot_resized, original_resized, full=True)

# Normalising the difference image for display
diff = (diff - diff.min()) / (diff.max() - diff.min())

print(f"SSIM Score: {ssim_score}")

# Creating a figure to display original, screenshot, and difference
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(20, 6))

ax1.imshow(original_resized, cmap='gray')
ax1.set_title('Original Image')
ax1.axis('off')

ax2.imshow(screenshot_resized, cmap='gray')
ax2.set_title('Screenshot')
ax2.axis('off')

ax3.imshow(diff, cmap='viridis')
ax3.set_title('Difference (SSIM)')
ax3.axis('off')

plt.tight_layout()
plt.show()

# Saving the comparison figure
comparison_path = "image_comparison.png"
fig.savefig(comparison_path)
print(f"Comparison image saved as {comparison_path}")

# Displaying the comparison image in the notebook
display(Image.open(comparison_path))

In [ ]:
print("\nPlease evaluate the following criteria:")
scores, total_score = get_manual_scores()
rating = get_rating(total_score)

# Creating a figure to display original, screenshot, difference, and scores
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(20, 20))

ax1.imshow(original_resized, cmap='gray')
ax1.set_title('Original Image')
ax1.axis('off')

ax2.imshow(screenshot_resized, cmap='gray')
ax2.set_title('Screenshot')
ax2.axis('off')

ax3.imshow(diff, cmap='viridis')
ax3.set_title('Difference (SSIM)')
ax3.axis('off')

ax4.axis('off')
score_text = f"Evaluation Scores:\n\n"
for criterion, score in scores.items():
    score_text += f"{criterion}: {score}/3\n"
score_text += f"\nTotal Score: {total_score}/15\n"
score_text += f"Rating: {rating}\n"
score_text += f"\nSSIM Score: {ssim_score:.4f}"
ax4.text(0.1, 0.5, score_text, fontsize=12, va='center')

plt.tight_layout()
plt.show()

# Saving the comparison figure
comparison_path = "image_comparison_with_scores.png"
fig.savefig(comparison_path)
print(f"Comparison image with scores saved as {comparison_path}")

# Displaying the comparison image in the notebook
display(Image.open(comparison_path))